# Preamble

In [15]:
import numpy as np
import bqplot.pyplot as plt
from bqplot import *
import random 

In [9]:
def make_plot(x, y, title, x_label, y_labels, y_sides=('left', 'right'), y_colors=CATEGORY10, y_grid_lines=('solid', 'dashed')):
    x_sc = LinearScale()
    y_scs = [LinearScale() for _ in y]

    ax_x = Axis(label=x_label, label_offset='4em', scale=x_sc)
    ax_ys = [Axis(label=label, label_offset='4em', scale=y_sc,
                  side=side,
                  color=color,
                  grid_lines=grid_lines)
             for y_sc, label, side, color, grid_lines in zip(y_scs, y_labels, y_sides, y_colors, y_grid_lines)]

    lines = [Lines(labels=[label],
                   x=x,
                   y=y_data,
                   scales={'x': x_sc, 'y': y_sc},
                   colors=[color])
             for y_sc, y_data, label, color in zip(y_scs, y, y_labels, y_colors)]

    plt.figure(title=title, axes=[ax_x, *ax_ys], marks=lines,
               fig_margin={'top': 60, 'bottom': 60, 'left': 70, 'right': 70})
    plt.show()

# Stakers

In this simulation, stakers claim WIZ from their staked GNO every day and choose to lock all of their GNO with 50/50 odds. The lock period is 30 days, which gives the supply graph a rough periodicity of around 31 days (given the locking all of GNO with 50/50 odds and the adding up of all the roughly sawtooth graphs of individual WIZ holdings). All the stakers are given equal GNO at the start.

In [10]:
class Staker(object):
    def __init__(self, GNO):
        self.GNO = GNO
        self.isLocked = False
        self.LockedInDays = 0
        self.LockingDate = 0
        self.WIZ = 0
        self.lastWIZWithdraw = 0
        self.IssueRate = 0

    def withdrawWIZ(self,today):
        global SupplyWIZ
        if today>self.lastWIZWithdraw and self.isLocked:
            SupplyWIZ[today]+=(today-self.lastWIZWithdraw)*self.IssueRate
            self.lastWIZWithdraw = today
        
    def checkUnlock(self, today):
        global LockedGNO
        self.withdrawWIZ(today)
        if today>self.LockingDate+self.LockedInDays and self.isLocked:
            LockedGNO[today]-=self.GNO
            self.LockedInDays=0
            self.isLocked= False
        
    def lockGNO(self, today):
        global LockedGNOAtStart
        global LockedGNO
        global SupplyWIZ
        self.checkUnlock(today)
        if not self.isLocked:
            ir=max(0,((CurrentFeesInWIZ[today]+CurrentFeesInETH[today])*12*min(1,CurrentFeesInETH[today]/CurrentFeesInWIZ[today])-SupplyWIZ[today])/30)
            r=np.random.rand()
            if r>0.5:# and ir>(CurrentFeesInWIZ[today])/30: 
                LockedGNOAtStart=max(0,LockedGNOAtStart-self.GNO)
                SupplyWIZ[today]+=ir*self.GNO/(self.GNO+LockedGNO[today]+LockedGNOAtStart)*30*1/3
                self.IssueRate=ir*self.GNO/(self.GNO+LockedGNO[today]+LockedGNOAtStart)*2/3
                self.LockedInDays=30
                self.lastWIZWithdraw=today
                self.LockingDate=today
                self.isLocked=True
                LockedGNO[today]+=self.GNO

SEED = 0
np.random.seed(SEED)
NrParticipants = 100
SupplyGNO = 1000000000
Participants = [Staker(SupplyGNO/NrParticipants) for _ in range(NrParticipants)]
NR_OF_DAYS = 1000

LockedGNO = [0]
LockedGNOAtStart = SupplyGNO;
CurrentFeesInWIZ = np.maximum(0, np.concatenate([[7000], 7000 + np.cumsum(15 * (np.random.rand(NR_OF_DAYS) - 0.5))]))
CurrentFeesInETH = np.maximum(0, np.concatenate([[3000], 3000 + np.cumsum(15 * (np.random.rand(NR_OF_DAYS) - 0.5))]))
CurrentFeesCollected = CurrentFeesInWIZ + CurrentFeesInETH
SupplyWIZ=[CurrentFeesCollected[0]*43/10]            
    
            
for CurrentDay in range(NR_OF_DAYS):
    for p in Participants:
        p.withdrawWIZ(CurrentDay)
        p.lockGNO(CurrentDay)
    SupplyWIZ.append(SupplyWIZ[CurrentDay]-CurrentFeesInWIZ[CurrentDay]/30)
    if SupplyWIZ[CurrentDay+1]<0:
        SupplyWIZ[CurrentDay+1]=0
    LockedGNO.append(LockedGNO[CurrentDay])

make_plot(range(NR_OF_DAYS+1), [CurrentFeesInWIZ, CurrentFeesInETH],
          title='Simulation: Paid fees in WIZ/ETH given random seed '+str(SEED),
          x_label='Time (days)', y_labels=['Fees in WIZ', 'Fees in ETH'])

make_plot(range(NR_OF_DAYS+1), [SupplyWIZ, CurrentFeesInETH / CurrentFeesInWIZ],
          title='Simulation: Supply reaction to ETH/WIZ ratio of paid fees',
          x_label='Time (days)', y_labels=['Supply of WIZ', 'ETH/WIZ ratio of paid fees'],
          y_colors=CATEGORY10[2:])

VBox(children=(Figure(axes=[Axis(label='Time (days)', label_offset='4em', scale=LinearScale()), Axis(color='#1f77b4', label='Fees in WIZ', label_offset='4em', scale=LinearScale(), side='left'), Axis(color='#ff7f0e', grid_lines='dashed', label='Fees in ETH', label_offset='4em', scale=LinearScale(), side='right')], fig_margin={'top': 60, 'left': 70, 'bottom': 60, 'right': 70}, layout=Layout(min_width='125px'), marks=[Lines(colors=['#1f77b4'], interactions={'hover': 'tooltip'}, labels=['Fees in WIZ'], scales={'y': LinearScale(), 'x': LinearScale()}, scales_metadata={'color': {'dimension': 'color'}, 'y': {'dimension': 'y', 'orientation': 'vertical'}, 'x': {'dimension': 'x', 'orientation': 'horizontal'}}, tooltip_style={'opacity': 0.9}, x=array([   0,    1,    2, ...,  998,  999, 1000]), y=array([ 7000.        ,  7000.73220256,  7003.96004305, ...,  6940.23620015,
        6936.16589842,  6938.82301558])), Lines(colors=['#ff7f0e'], interactions={'hover': 'tooltip'}, labels=['Fees in ETH'], scales={'y': LinearScale(), 'x': LinearScale()}, scales_metadata={'color': {'dimension': 'color'}, 'y': {'dimension': 'y', 'orientation': 'vertical'}, 'x': {'dimension': 'x', 'orientation': 'horizontal'}}, tooltip_style={'opacity': 0.9}, x=array([   0,    1,    2, ...,  998,  999, 1000]), y=array([ 3000.        ,  3001.39320406,  2994.0441595 , ...,  3168.06524214,
        3170.22320269,  3170.24979965]))], scale_x=LinearScale(allow_padding=False, max=1.0, min=0.0), scale_y=LinearScale(allow_padding=False, max=1.0, min=0.0), title='Simulation: Paid fees in WIZ/ETH given random seed 0'), Toolbar(figure=Figure(axes=[Axis(label='Time (days)', label_offset='4em', scale=LinearScale()), Axis(color='#1f77b4', label='Fees in WIZ', label_offset='4em', scale=LinearScale(), side='left'), Axis(color='#ff7f0e', grid_lines='dashed', label='Fees in ETH', label_offset='4em', scale=LinearScale(), side='right')], fig_margin={'top': 60, 'left': 70, 'bottom': 60, 'right': 70}, layout=Layout(min_width='125px'), marks=[Lines(colors=['#1f77b4'], interactions={'hover': 'tooltip'}, labels=['Fees in WIZ'], scales={'y': LinearScale(), 'x': LinearScale()}, scales_metadata={'color': {'dimension': 'color'}, 'y': {'dimension': 'y', 'orientation': 'vertical'}, 'x': {'dimension': 'x', 'orientation': 'horizontal'}}, tooltip_style={'opacity': 0.9}, x=array([   0,    1,    2, ...,  998,  999, 1000]), y=array([ 7000.        ,  7000.73220256,  7003.96004305, ...,  6940.23620015,
        6936.16589842,  6938.82301558])), Lines(colors=['#ff7f0e'], interactions={'hover': 'tooltip'}, labels=['Fees in ETH'], scales={'y': LinearScale(), 'x': LinearScale()}, scales_metadata={'color': {'dimension': 'color'}, 'y': {'dimension': 'y', 'orientation': 'vertical'}, 'x': {'dimension': 'x', 'orientation': 'horizontal'}}, tooltip_style={'opacity': 0.9}, x=array([   0,    1,    2, ...,  998,  999, 1000]), y=array([ 3000.        ,  3001.39320406,  2994.0441595 , ...,  3168.06524214,
        3170.22320269,  3170.24979965]))], scale_x=LinearScale(allow_padding=False, max=1.0, min=0.0), scale_y=LinearScale(allow_padding=False, max=1.0, min=0.0), title='Simulation: Paid fees in WIZ/ETH given random seed 0'))))

VBox(children=(Figure(axes=[Axis(label='Time (days)', label_offset='4em', scale=LinearScale()), Axis(color='#2ca02c', label='Supply of WIZ', label_offset='4em', scale=LinearScale(), side='left'), Axis(color='#d62728', grid_lines='dashed', label='ETH/WIZ ratio of paid fees', label_offset='4em', scale=LinearScale(), side='right')], fig_margin={'top': 60, 'left': 70, 'bottom': 60, 'right': 70}, layout=Layout(min_width='125px'), marks=[Lines(colors=['#2ca02c'], interactions={'hover': 'tooltip'}, labels=['Supply of WIZ'], scales={'y': LinearScale(), 'x': LinearScale()}, scales_metadata={'color': {'dimension': 'color'}, 'y': {'dimension': 'y', 'orientation': 'vertical'}, 'x': {'dimension': 'x', 'orientation': 'horizontal'}}, tooltip_style={'opacity': 0.9}, x=array([   0,    1,    2, ...,  998,  999, 1000]), y=array([ 44175.8502175 ,  44688.15373707,  44908.57800452, ...,
        48710.28695234,  48678.55595183,  48447.35042188])), Lines(colors=['#d62728'], interactions={'hover': 'tooltip'}, labels=['ETH/WIZ ratio of paid fees'], scales={'y': LinearScale(), 'x': LinearScale()}, scales_metadata={'color': {'dimension': 'color'}, 'y': {'dimension': 'y', 'orientation': 'vertical'}, 'x': {'dimension': 'x', 'orientation': 'horizontal'}}, tooltip_style={'opacity': 0.9}, x=array([   0,    1,    2, ...,  998,  999, 1000]), y=array([ 0.42857143,  0.42872561,  0.42747876, ...,  0.45647801,
        0.457057  ,  0.45688581]))], scale_x=LinearScale(allow_padding=False, max=1.0, min=0.0), scale_y=LinearScale(allow_padding=False, max=1.0, min=0.0), title='Simulation: Supply reaction to ETH/WIZ ratio of paid fees'), Toolbar(figure=Figure(axes=[Axis(label='Time (days)', label_offset='4em', scale=LinearScale()), Axis(color='#2ca02c', label='Supply of WIZ', label_offset='4em', scale=LinearScale(), side='left'), Axis(color='#d62728', grid_lines='dashed', label='ETH/WIZ ratio of paid fees', label_offset='4em', scale=LinearScale(), side='right')], fig_margin={'top': 60, 'left': 70, 'bottom': 60, 'right': 70}, layout=Layout(min_width='125px'), marks=[Lines(colors=['#2ca02c'], interactions={'hover': 'tooltip'}, labels=['Supply of WIZ'], scales={'y': LinearScale(), 'x': LinearScale()}, scales_metadata={'color': {'dimension': 'color'}, 'y': {'dimension': 'y', 'orientation': 'vertical'}, 'x': {'dimension': 'x', 'orientation': 'horizontal'}}, tooltip_style={'opacity': 0.9}, x=array([   0,    1,    2, ...,  998,  999, 1000]), y=array([ 44175.8502175 ,  44688.15373707,  44908.57800452, ...,
        48710.28695234,  48678.55595183,  48447.35042188])), Lines(colors=['#d62728'], interactions={'hover': 'tooltip'}, labels=['ETH/WIZ ratio of paid fees'], scales={'y': LinearScale(), 'x': LinearScale()}, scales_metadata={'color': {'dimension': 'color'}, 'y': {'dimension': 'y', 'orientation': 'vertical'}, 'x': {'dimension': 'x', 'orientation': 'horizontal'}}, tooltip_style={'opacity': 0.9}, x=array([   0,    1,    2, ...,  998,  999, 1000]), y=array([ 0.42857143,  0.42872561,  0.42747876, ...,  0.45647801,
        0.457057  ,  0.45688581]))], scale_x=LinearScale(allow_padding=False, max=1.0, min=0.0), scale_y=LinearScale(allow_padding=False, max=1.0, min=0.0), title='Simulation: Supply reaction to ETH/WIZ ratio of paid fees'))))

In [25]:
class Staker(object):
    def __init__(self, GNO):
        self.GNO = GNO
        self.isLocked = False
        self.LockedInDays = 0
        self.LockingDate = 0
        self.WIZ = 0
        self.lastWIZWithdraw = 0
        self.IssueRate = 0

    def withdrawWIZ(self,today):
        global SupplyWIZ
        if today>self.lastWIZWithdraw and self.isLocked:
            SupplyWIZ[today]+=(today-self.lastWIZWithdraw)*self.IssueRate
            self.lastWIZWithdraw = today
        
    def checkUnlock(self, today):
        global LockedGNO
        self.withdrawWIZ(today)
        if today>self.LockingDate+self.LockedInDays and self.isLocked:
            LockedGNO[today]-=self.GNO
            self.LockedInDays=0
            self.isLocked= False
        
    def lockGNO(self, today):
        global LockedGNOAtStart
        global LockedGNO
        global SupplyWIZ
        self.checkUnlock(today)
        if not self.isLocked:
            if SumOfETHConsumed30Days[today]/SumOfWIZConsumed30Days[today]<1/9:
                ir=max(0,((SumOfWIZConsumed30Days[today]+SumOfETHConsumed30Days[today])*180*1/9-SupplyWIZ[today])/30)
            else:
                ir=max(0,((SumOfWIZConsumed30Days[today]+SumOfETHConsumed30Days[today])*180*min(1/2,SumOfETHConsumed30Days[today]/SumOfWIZConsumed30Days[today])-SupplyWIZ[today])/30)
                    
            r=np.random.rand()
            if r>0.5:# and ir>(CurrentFeesInWIZ[today])/30: 
                LockedGNOAtStart=max(0,LockedGNOAtStart-self.GNO)
                SupplyWIZ[today]+=ir*self.GNO/(self.GNO+LockedGNO[today]+LockedGNOAtStart)*30*1/3
                self.IssueRate=ir*self.GNO/(self.GNO+LockedGNO[today]+LockedGNOAtStart)*2/3
                self.LockedInDays=30
                self.lastWIZWithdraw=today
                self.LockingDate=today
                self.isLocked=True
                LockedGNO[today]+=self.GNO

def getNewFeesCollected(CurrentDay):
        global SumOfWIZConsumed30Days
        global SumOfETHConsumed30Days
        
        CurrentFeesInWIZ.append(CurrentFeesInWIZ[CurrentDay]+((random.random()-1/2)*30))
        #negative supply not allowed
        if SupplyWIZ[CurrentDay]-CurrentFeesInWIZ[CurrentDay+1]<0:
            CurrentFeesInWIZ[CurrentDay+1]=SupplyWIZ[CurrentDay]
        #check that the sum is substracted by initialized days in case we are in the first 30 days    
        if CurrentDay<=30:
            SumOfWIZConsumed30Days.append(SumOfWIZConsumed30Days[CurrentDay]-CurrentFeesInWIZ[0]+CurrentFeesInWIZ[CurrentDay+1])
        else:
            SumOfWIZConsumed30Days.append(SumOfWIZConsumed30Days[CurrentDay]-CurrentFeesInWIZ[CurrentDay-30]+CurrentFeesInWIZ[CurrentDay+1])
        
        CurrentFeesInETH.append(CurrentFeesInETH[CurrentDay]+((random.random()-1/2)*30))
        if CurrentDay<=30:
            SumOfETHConsumed30Days.append(SumOfETHConsumed30Days[CurrentDay]-CurrentFeesInETH[0]+CurrentFeesInETH[CurrentDay+1])
        else:
            SumOfETHConsumed30Days.append(SumOfETHConsumed30Days[CurrentDay]-CurrentFeesInETH[CurrentDay-30]+CurrentFeesInETH[CurrentDay+1])
     
 

                
SEED = 0
np.random.seed(SEED)
NrParticipants = 100
SupplyGNO = 1000000000
Participants = [Staker(SupplyGNO/NrParticipants) for _ in range(NrParticipants)]
NR_OF_DAYS = 1000

LockedGNO = [0]
LockedGNOAtStart = SupplyGNO;
CurrentFeesInWIZ = [7000]
CurrentFeesInETH = [3000]
CurrentFeesCollected = CurrentFeesInWIZ + CurrentFeesInETH
SupplyWIZ=[CurrentFeesCollected[0]*30*20]            
SumOfWIZConsumed30Days=[CurrentFeesInWIZ[0]*30]
SumOfETHConsumed30Days=[CurrentFeesInETH[0]*30]
           
for CurrentDay in range(NR_OF_DAYS):
    for p in Participants:
        p.withdrawWIZ(CurrentDay)
        p.lockGNO(CurrentDay)
    getNewFeesCollected(CurrentDay)
    CurrentFeesCollected.append(CurrentFeesInWIZ[CurrentDay+1]+CurrentFeesInETH[CurrentDay+1])
    SupplyWIZ.append(SupplyWIZ[CurrentDay]-CurrentFeesInWIZ[CurrentDay])
    LockedGNO.append(LockedGNO[CurrentDay])

make_plot(range(NR_OF_DAYS+1), [CurrentFeesInWIZ, CurrentFeesInETH],
          title='Simulation: Paid fees in WIZ/ETH given random seed '+str(SEED),
          x_label='Time (days)', y_labels=['Fees in WIZ', 'Fees in ETH'])

make_plot(range(NR_OF_DAYS+1), [SupplyWIZ, np.array(CurrentFeesInETH, dtype=np.float)/np.array(CurrentFeesInWIZ, dtype=np.float)],
          title='Simulation: Supply reaction to ETH/WIZ ratio of paid fees',
          x_label='Time (days)', y_labels=['Supply of WIZ', 'ETH/WIZ ratio of paid fees'],
          y_colors=CATEGORY10[2:])

VBox(children=(Figure(axes=[Axis(label='Time (days)', label_offset='4em', scale=LinearScale()), Axis(color='#1f77b4', label='Fees in WIZ', label_offset='4em', scale=LinearScale(), side='left'), Axis(color='#ff7f0e', grid_lines='dashed', label='Fees in ETH', label_offset='4em', scale=LinearScale(), side='right')], fig_margin={'top': 60, 'left': 70, 'bottom': 60, 'right': 70}, layout=Layout(min_width='125px'), marks=[Lines(colors=['#1f77b4'], interactions={'hover': 'tooltip'}, labels=['Fees in WIZ'], scales={'y': LinearScale(), 'x': LinearScale()}, scales_metadata={'color': {'dimension': 'color'}, 'y': {'dimension': 'y', 'orientation': 'vertical'}, 'x': {'dimension': 'x', 'orientation': 'horizontal'}}, tooltip_style={'opacity': 0.9}, x=array([   0,    1,    2, ...,  998,  999, 1000]), y=array([ 7000.        ,  6998.48617135,  6989.61041455, ...,  6739.09296363,
        6733.49804412,  6735.40325256])), Lines(colors=['#ff7f0e'], interactions={'hover': 'tooltip'}, labels=['Fees in ETH'], scales={'y': LinearScale(), 'x': LinearScale()}, scales_metadata={'color': {'dimension': 'color'}, 'y': {'dimension': 'y', 'orientation': 'vertical'}, 'x': {'dimension': 'x', 'orientation': 'horizontal'}}, tooltip_style={'opacity': 0.9}, x=array([   0,    1,    2, ...,  998,  999, 1000]), y=array([ 3000.        ,  2998.53621648,  2983.62996757, ...,  2666.65427778,
        2672.97316503,  2678.41997146]))], scale_x=LinearScale(allow_padding=False, max=1.0, min=0.0), scale_y=LinearScale(allow_padding=False, max=1.0, min=0.0), title='Simulation: Paid fees in WIZ/ETH given random seed 0'), Toolbar(figure=Figure(axes=[Axis(label='Time (days)', label_offset='4em', scale=LinearScale()), Axis(color='#1f77b4', label='Fees in WIZ', label_offset='4em', scale=LinearScale(), side='left'), Axis(color='#ff7f0e', grid_lines='dashed', label='Fees in ETH', label_offset='4em', scale=LinearScale(), side='right')], fig_margin={'top': 60, 'left': 70, 'bottom': 60, 'right': 70}, layout=Layout(min_width='125px'), marks=[Lines(colors=['#1f77b4'], interactions={'hover': 'tooltip'}, labels=['Fees in WIZ'], scales={'y': LinearScale(), 'x': LinearScale()}, scales_metadata={'color': {'dimension': 'color'}, 'y': {'dimension': 'y', 'orientation': 'vertical'}, 'x': {'dimension': 'x', 'orientation': 'horizontal'}}, tooltip_style={'opacity': 0.9}, x=array([   0,    1,    2, ...,  998,  999, 1000]), y=array([ 7000.        ,  6998.48617135,  6989.61041455, ...,  6739.09296363,
        6733.49804412,  6735.40325256])), Lines(colors=['#ff7f0e'], interactions={'hover': 'tooltip'}, labels=['Fees in ETH'], scales={'y': LinearScale(), 'x': LinearScale()}, scales_metadata={'color': {'dimension': 'color'}, 'y': {'dimension': 'y', 'orientation': 'vertical'}, 'x': {'dimension': 'x', 'orientation': 'horizontal'}}, tooltip_style={'opacity': 0.9}, x=array([   0,    1,    2, ...,  998,  999, 1000]), y=array([ 3000.        ,  2998.53621648,  2983.62996757, ...,  2666.65427778,
        2672.97316503,  2678.41997146]))], scale_x=LinearScale(allow_padding=False, max=1.0, min=0.0), scale_y=LinearScale(allow_padding=False, max=1.0, min=0.0), title='Simulation: Paid fees in WIZ/ETH given random seed 0'))))

VBox(children=(Figure(axes=[Axis(label='Time (days)', label_offset='4em', scale=LinearScale()), Axis(color='#2ca02c', label='Supply of WIZ', label_offset='4em', scale=LinearScale(), side='left'), Axis(color='#d62728', grid_lines='dashed', label='ETH/WIZ ratio of paid fees', label_offset='4em', scale=LinearScale(), side='right')], fig_margin={'top': 60, 'left': 70, 'bottom': 60, 'right': 70}, layout=Layout(min_width='125px'), marks=[Lines(colors=['#2ca02c'], interactions={'hover': 'tooltip'}, labels=['Supply of WIZ'], scales={'y': LinearScale(), 'x': LinearScale()}, scales_metadata={'color': {'dimension': 'color'}, 'y': {'dimension': 'y', 'orientation': 'vertical'}, 'x': {'dimension': 'x', 'orientation': 'horizontal'}}, tooltip_style={'opacity': 0.9}, x=array([   0,    1,    2, ...,  998,  999, 1000]), y=array([  7058925.0549718 ,   8908373.4253137 ,   9709713.01075448, ...,
        20981128.41404704,  20974389.32108341,  20967655.82303929])), Lines(colors=['#d62728'], interactions={'hover': 'tooltip'}, labels=['ETH/WIZ ratio of paid fees'], scales={'y': LinearScale(), 'x': LinearScale()}, scales_metadata={'color': {'dimension': 'color'}, 'y': {'dimension': 'y', 'orientation': 'vertical'}, 'x': {'dimension': 'x', 'orientation': 'horizontal'}}, tooltip_style={'opacity': 0.9}, x=array([   0,    1,    2, ...,  998,  999, 1000]), y=array([ 0.42857143,  0.42845497,  0.42686642, ...,  0.39569929,
        0.3969665 ,  0.3976629 ]))], scale_x=LinearScale(allow_padding=False, max=1.0, min=0.0), scale_y=LinearScale(allow_padding=False, max=1.0, min=0.0), title='Simulation: Supply reaction to ETH/WIZ ratio of paid fees'), Toolbar(figure=Figure(axes=[Axis(label='Time (days)', label_offset='4em', scale=LinearScale()), Axis(color='#2ca02c', label='Supply of WIZ', label_offset='4em', scale=LinearScale(), side='left'), Axis(color='#d62728', grid_lines='dashed', label='ETH/WIZ ratio of paid fees', label_offset='4em', scale=LinearScale(), side='right')], fig_margin={'top': 60, 'left': 70, 'bottom': 60, 'right': 70}, layout=Layout(min_width='125px'), marks=[Lines(colors=['#2ca02c'], interactions={'hover': 'tooltip'}, labels=['Supply of WIZ'], scales={'y': LinearScale(), 'x': LinearScale()}, scales_metadata={'color': {'dimension': 'color'}, 'y': {'dimension': 'y', 'orientation': 'vertical'}, 'x': {'dimension': 'x', 'orientation': 'horizontal'}}, tooltip_style={'opacity': 0.9}, x=array([   0,    1,    2, ...,  998,  999, 1000]), y=array([  7058925.0549718 ,   8908373.4253137 ,   9709713.01075448, ...,
        20981128.41404704,  20974389.32108341,  20967655.82303929])), Lines(colors=['#d62728'], interactions={'hover': 'tooltip'}, labels=['ETH/WIZ ratio of paid fees'], scales={'y': LinearScale(), 'x': LinearScale()}, scales_metadata={'color': {'dimension': 'color'}, 'y': {'dimension': 'y', 'orientation': 'vertical'}, 'x': {'dimension': 'x', 'orientation': 'horizontal'}}, tooltip_style={'opacity': 0.9}, x=array([   0,    1,    2, ...,  998,  999, 1000]), y=array([ 0.42857143,  0.42845497,  0.42686642, ...,  0.39569929,
        0.3969665 ,  0.3976629 ]))], scale_x=LinearScale(allow_padding=False, max=1.0, min=0.0), scale_y=LinearScale(allow_padding=False, max=1.0, min=0.0), title='Simulation: Supply reaction to ETH/WIZ ratio of paid fees'))))

In [4]:
SEED = 0
np.random.seed(SEED)
NrParticipants = 100
SupplyGNO = 1000000000
Participants = [Staker(SupplyGNO/NrParticipants) for _ in range(NrParticipants)]
NR_OF_DAYS = 1000

LockedGNO = [0]
LockedGNOAtStart = SupplyGNO;
CurrentFeesInWIZ = np.maximum(0, np.concatenate([[7000], 7000 + np.cumsum(15 * (np.random.rand(NR_OF_DAYS) - 0.5))])) * \
    np.power(.999, np.linspace(0, NR_OF_DAYS, NR_OF_DAYS + 1))
CurrentFeesInETH = np.maximum(0, np.concatenate([[3000], 3000 + np.cumsum(15 * (np.random.rand(NR_OF_DAYS) - 0.5))])) * \
    np.power(.999, np.linspace(0, NR_OF_DAYS, NR_OF_DAYS + 1))
CurrentFeesCollected = CurrentFeesInWIZ + CurrentFeesInETH
SupplyWIZ=[CurrentFeesCollected[0]*43/10]            
    
            
for CurrentDay in range(NR_OF_DAYS):
    for p in Participants:
        p.withdrawWIZ(CurrentDay)
        p.lockGNO(CurrentDay)
    SupplyWIZ.append(SupplyWIZ[CurrentDay]-CurrentFeesInWIZ[CurrentDay]/30)
    if SupplyWIZ[CurrentDay+1]<0:
        SupplyWIZ[CurrentDay+1]=0
    LockedGNO.append(LockedGNO[CurrentDay])

make_plot(range(NR_OF_DAYS+1), [CurrentFeesInWIZ, CurrentFeesInETH],
          title='Simulation: Declining paid fees in WIZ/ETH with random seed '+str(SEED),
          x_label='Time (days)', y_labels=['Fees in WIZ', 'Fees in ETH'])

make_plot(range(NR_OF_DAYS+1), [SupplyWIZ, CurrentFeesInETH / CurrentFeesInWIZ],
          title='Simulation: Supply reaction to ETH/WIZ ratio of declining paid fees',
          x_label='Time (days)', y_labels=['Supply of WIZ', 'ETH/WIZ ratio of paid fees'],
          y_colors=CATEGORY10[2:])

A Jupyter Widget

A Jupyter Widget

In [5]:
SEED = 0
np.random.seed(SEED)
NrParticipants = 100
SupplyGNO = 1000000000
Participants = [Staker(SupplyGNO/NrParticipants) for _ in range(NrParticipants)]
NR_OF_DAYS = 1000

LockedGNO = [0]
LockedGNOAtStart = SupplyGNO;
CurrentFeesInWIZ = np.maximum(0, np.concatenate([[7000], 7000 + np.cumsum(15 * (np.random.rand(NR_OF_DAYS) - 0.5))])) * \
    np.power(.995, np.linspace(0, NR_OF_DAYS, NR_OF_DAYS + 1))
CurrentFeesInETH = np.maximum(0, np.concatenate([[3000], 3000 + np.cumsum(15 * (np.random.rand(NR_OF_DAYS) - 0.5))]))
CurrentFeesCollected = CurrentFeesInWIZ + CurrentFeesInETH
SupplyWIZ=[CurrentFeesCollected[0]*43/10]            
    
            
for CurrentDay in range(NR_OF_DAYS):
    for p in Participants:
        p.withdrawWIZ(CurrentDay)
        p.lockGNO(CurrentDay)
    SupplyWIZ.append(SupplyWIZ[CurrentDay]-CurrentFeesInWIZ[CurrentDay]/30)
    if SupplyWIZ[CurrentDay+1]<0:
        SupplyWIZ[CurrentDay+1]=0
    LockedGNO.append(LockedGNO[CurrentDay])

make_plot(range(NR_OF_DAYS+1), [CurrentFeesInWIZ, CurrentFeesInETH],
          title='Simulation: Declining paid fees in WIZ/ETH with random seed '+str(SEED),
          x_label='Time (days)', y_labels=['Fees in WIZ', 'Fees in ETH'])

make_plot(range(NR_OF_DAYS+1), [SupplyWIZ, CurrentFeesInETH / CurrentFeesInWIZ],
          title='Simulation: Supply reaction to ETH/WIZ ratio of declining paid fees',
          x_label='Time (days)', y_labels=['Supply of WIZ', 'ETH/WIZ ratio of paid fees'],
          y_colors=CATEGORY10[2:])

A Jupyter Widget

A Jupyter Widget